In [7]:
import pandas as pd
import regex as re

variables_df = pd.read_excel('input/coded_dataset/variables.xlsx')
variables_overview_df = pd.read_csv('input/variables_overview.csv', sep='\t')

def is_identifier(variable_name: str) -> bool:
    '''Returns True if the variable name refers to an identifier, False otherwise'''
    return variable_name.startswith('Identifier') or variable_name.endswith('ID') or variable_name.endswith('IDs') or variable_name.endswith('hash')

def elvis(value, default):
    '''Returns the value if it is not None or NA, default otherwise'''
    if pd.isna(value):
        return default
    return value

In [8]:
def get_nominal_stat(df, variable_id, variable_name: str):

    series_bg = df[df['CG'] == False][variable_name]
    series_cg = df[df['CG'] == True][variable_name]

    tex = f"""{variable_id} & {series_bg.count()} & {series_cg.count()} & {series_bg.nunique()} & {series_cg.nunique() if series_cg.count() > 0 else '-'} \\\\"""

    return tex

def get_numeric_stat(df, variable_id, variable_name):
    series_bg: pd.Series = df[df['CG'] == False][variable_name]
    series_cg: pd.Series = df[df['CG'] == True][variable_name]

    count_bg = series_bg.count()
    count_cg = series_cg.count()
    mean_bg = elvis(round(series_bg.mean(), 2), '-')
    mean_cg = elvis(round(series_cg.mean(), 2), '-')
    std_bg = elvis(round(series_bg.std(), 2), '-')
    std_cg = elvis(round(series_cg.std(), 2), '-')
    
    tex = f"""{variable_id} & {count_bg} & {count_cg} & {mean_bg} & {mean_cg} & {std_bg} & {std_cg} \\\\"""

    return tex

def get_list_stat(df, variable_id, variable_name):
    series_bg: pd.Series = df[df['CG'] == False][variable_name]
    series_cg: pd.Series = df[df['CG'] == True][variable_name]

    series_bg = series_bg.dropna().astype(str).apply(eval)
    series_cg = series_cg.dropna().astype(str).apply(eval)

    count_non_empty_bg = series_bg.apply(lambda x: len(x) > 0).sum()
    count_non_empty_cg = series_cg.apply(lambda x: len(x) > 0).sum()
    sum_labels_bg = series_bg.apply(len).sum()
    sum_labels_cg = series_cg.apply(len).sum() if count_non_empty_cg > 0 else '-'
    unique_labels_bg = len(set(label for sublist in series_bg for label in sublist))
    unique_labels_cg = len(set(label for sublist in series_cg for label in sublist)) if count_non_empty_cg > 0 else '-'

    tex = f"""{variable_id} & {count_non_empty_bg} & {count_non_empty_cg} & {sum_labels_bg} & {sum_labels_cg} & {unique_labels_bg} & {unique_labels_cg} \\\\"""

    return tex

In [ ]:
results_nominal = []
results_numeric = []
results_list_of_nominals = []
prev_variable_source = None


for variable_id, variable_name, variable_scale, variable_aggregation in variables_overview_df[['ID', 'Name', 'Scale', 'Aggregation']].values:
    if not pd.notna(variable_id) or is_identifier(variable_name):
        continue
    
    variable_aggregation = variable_aggregation if pd.notna(variable_aggregation) else None
    variable_name = f'{variable_id} - {variable_name}' + (f' [{variable_aggregation}]' if variable_aggregation else '')
    variable_scale = str(variable_scale).lower()

    variable_source = re.match(r'[A-Za-z]+', variable_id).group()
    prev_variable_source = prev_variable_source or variable_source
    midrule = '\\midrule'
    if variable_source != prev_variable_source:
        if results_nominal[-1] != midrule:
            results_nominal.append(midrule)
        if results_numeric[-1] != midrule:
            results_numeric.append(midrule)
        if results_list_of_nominals[-1] != midrule:
            results_list_of_nominals.append(midrule)
    

    if variable_scale in ['nominal', 'boolean']:
        results_nominal.append(get_nominal_stat(variables_df, variable_id, variable_name))
    elif variable_scale in ['integer', 'float']:
        results_numeric.append(get_numeric_stat(variables_df, variable_id, variable_name))
    elif variable_scale in ['list of nominals']:
        results_list_of_nominals.append(get_list_stat(variables_df, variable_id, variable_name))
    else:
        raise ValueError(f'Unknown scale: {variable_scale}')

    prev_variable_source = variable_source

tex_nominal = f"""
\\begin{{table}}[h]
\\centering
\\begin{{tabular}}[t]{{c|cc|cc}}
\\toprule
\\multirow{2}{{*}}{{ID}} & \\multicolumn{{2}}{{c|}}{{Valid Instances}} & \\multicolumn{{2}}{{c}}{{\\#Unique Labels}} \\\\
\\cmidrule{{2-5}}
& BG & CG & BG & CG \\\\\\midrule
{'\n'.join(results_nominal)}
\\bottomrule
\\end{{tabular}}
\\caption{{Distribution of nominal variables. A dash (-) indicates that the variable is not present in the sample group.}}
\\label{{tbl:data-dist-nom}}
\\end{{table}}
"""

tex_numeric = f"""
\\begin{{table}}[h]
\\centering
\\begin{{tabular}}{{c|cc|cc|cc}}
\\toprule
\\multirow{2}{{*}}{{ID}} & \\multicolumn{{2}}{{c|}}{{Valid Instances}} & \\multicolumn{{2}}{{c|}}{{Mean}} & \\multicolumn{{2}}{{c}}{{Standard Deviation}} \\\\
\\cmidrule{{2-7}}
& BG & CG & BG & CG & BG & CG \\\\\\midrule
{'\n'.join(results_numeric)}
\\bottomrule
\\end{{tabular}}
\\caption{{Distribution of numeric variables. A dash (-) indicates that the variable is not present in the sample group.}}
\\label{{tbl:data-dist-num}}
\\end{{table}}
"""

tex_list_of_nominals = f"""
\\begin{{table}}[h]
\\centering
\\begin{{tabular}}{{c|cc|cc|cc}}
\\toprule
\\multirow{{2}}{{*}}{{ID}} & \\multicolumn{{2}}{{c|}}{{Valid Instances}} & \\multicolumn{{2}}{{c|}}{{\\#Absolute Labels}} & \\multicolumn{{2}}{{c}}{{\\#Unique Labels}} \\\\
\\cmidrule{{2-7}}
& BG & CG & BG & CG & BG & CG \\\\\\midrule
{'\n'.join(results_list_of_nominals)}
\\bottomrule
\\end{{tabular}}
\\caption{{Distribution of list of nominal variables. A dash (-) indicates that the variable is not present in the sample group.}}
\\label{{tbl:data-dist-list}}
\\end{{table}}
"""

with open('output/results/data_distribution_tables.tex', 'w') as f:
    f.write(tex_nominal)
    f.write(tex_list_of_nominals)
    f.write(tex_numeric)


